## Validation RL-Element

### Import dpsimpy 

In [ ]:
import dpsimpy

### Parameters

In [ ]:
resistance = 10
inductance = 1e-3
capacitance = 5e-6

### Powerflow for Initialization

In [ ]:
sim_name_pf = 'VS_RLC1_PF'
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)

# Nodes
gnd_pf = dpsimpy.sp.SimNode.gnd
n1_pf  = dpsimpy.sp.SimNode('n1_pf')
n2_pf  = dpsimpy.sp.SimNode('n2_pf')
n3_pf  = dpsimpy.sp.SimNode('n3_pf')

# Components
vs_pf = dpsimpy.sp.ph1.VoltageSource('vs_pf', dpsimpy.LogLevel.debug)
vs_pf.set_parameters(V_ref=complex(10,0), f_src=50)
r1_pf = dpsimpy.sp.ph1.Resistor('r1_pf', dpsimpy.LogLevel.debug)
r1_pf.set_parameters(R=resistance)
c1_pf = dpsimpy.sp.ph1.Capacitor('c1_pf', dpsimpy.LogLevel.debug)
c1_pf.set_parameters(C=capacitance)
l1_pf = dpsimpy.sp.ph1.Inductor('l1_pf', dpsimpy.LogLevel.debug)
l1_pf.set_parameters(L=inductance)

# Connections
vs_pf.connect([gnd_pf, n1_pf])
r1_pf.connect([n2_pf, n1_pf])
l1_pf.connect([n3_pf, n2_pf])
c1_pf.connect([gnd_pf, n3_pf])

# Define system topology
system_pf = dpsimpy.SystemTopology(50, [n1_pf, n2_pf, n3_pf], [vs_pf, r1_pf, l1_pf, c1_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('n1.v', 'v', n1_pf)
logger_pf.log_attribute('n2.v', 'v', n2_pf)
logger_pf.log_attribute('n3.v', 'v', n3_pf)
logger_pf.log_attribute('r1.i_intf', 'i_intf', r1_pf)
logger_pf.log_attribute('l1.i_intf', 'i_intf', l1_pf)

sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_time_step(0.1)
sim_pf.set_final_time(0.5)
sim_pf.add_logger(logger_pf)
sim_pf.run()

In [ ]:
print(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0])
print(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0][0])
print(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0][0])

### DPsim DP - RLC circuit

In [ ]:
name = 'DP_VS_RLC'
dpsimpy.Logger.set_log_dir("logs/" + name)

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode('n1')
n1.set_initial_voltage(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0])
n2 = dpsimpy.dp.SimNode('n2')
n2.set_initial_voltage(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0])
n3 = dpsimpy.dp.SimNode('n3')
n3.set_initial_voltage(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0])

# Components
vs = dpsimpy.dp.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(10,0), f_src=50)
r1 = dpsimpy.dp.ph1.Resistor('r_1', dpsimpy.LogLevel.debug)
r1.R = resistance
l1 = dpsimpy.dp.ph1.Inductor('l_1', dpsimpy.LogLevel.debug)
l1.L = inductance
c1 = dpsimpy.dp.ph1.Capacitor('c1', dpsimpy.LogLevel.debug)
c1.set_parameters(C=capacitance)

# Connections
vs.connect([gnd, n1])
r1.connect([n2, n1])
l1.connect([n3, n2])
c1.connect([gnd, n3])

# Define system topology
system = dpsimpy.SystemTopology(50, [gnd, n1, n2, n3], [vs, r1, l1, c1])

# Logging
logger = dpsimpy.Logger(name)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('n3.v', 'v', n3)
logger.log_attribute('l1.i_intf', 'i_intf', l1)


sim = dpsimpy.Simulation(name)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_time_step(0.0001)
sim.set_final_time(0.02)
sim.add_logger(logger)
sim.run()

### Read Results

In [ ]:
# read Simulink log file

import matplotlib.pyplot as plt
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt
import os

# read EMT results
file_path = os.getcwd() + "/logs/" + name + "/" + name + ".csv"
ts_dpsim_dp = rt.read_timeseries_dpsim(file_path)

#convert to emt
ts_dpsim_dp_emt = ts.frequency_shift_list(ts_dpsim_dp, 50)

## DPsim DP - with RL element

In [ ]:
name2 = 'DP_VS_RLC2'
dpsimpy.Logger.set_log_dir("logs/" + name2)

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode('n1')
n1.set_initial_voltage(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0])
n2 = dpsimpy.dp.SimNode('n2')
n2.set_initial_voltage(sim_pf.get_idobj_attr(n3_pf.name(), 'v').get()[0])

# Components
vs = dpsimpy.dp.ph1.VoltageSource('vs', dpsimpy.LogLevel.debug)
vs.set_parameters(V_ref=complex(10,0), f_src=50)
rl = dpsimpy.dp.ph1.ResInductor('rl', dpsimpy.LogLevel.debug)
rl.set_parameters(R=resistance, L=inductance)
c1 = dpsimpy.dp.ph1.Capacitor('c1', dpsimpy.LogLevel.debug)
c1.set_parameters(C=capacitance)

# Connections
vs.connect([gnd, n1])
rl.connect([n2, n1])
c1.connect([gnd, n2])

# Define system topology
system = dpsimpy.SystemTopology(50, [gnd, n1, n2], [vs, rl, c1])

# Logging
logger = dpsimpy.Logger(name2)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('l1.i_intf', 'i_intf', rl)
logger.log_attribute('rl.v', 'v_intf', rl)

sim = dpsimpy.Simulation(name2)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_time_step(0.0001)
sim.set_final_time(0.02)
sim.add_logger(logger)
sim.run()

### Read Results

In [ ]:
# read EMT results
file_path = os.getcwd() + "/logs/" + name2 + "/" + name2 + ".csv"
ts_dpsim_dp2 = rt.read_timeseries_dpsim(file_path)

#convert to emt
ts_dpsim_dp_emt2 = ts.frequency_shift_list(ts_dpsim_dp2, 50)

### Plot Results

In [ ]:
ts_dpsim_dp_emt["n3.v_shift"].label = 'vc - RLC circuit'
ts_dpsim_dp_emt["l1.i_intf_shift"].label = 'i - RLC circuit'

ts_dpsim_dp_emt2["n2.v_shift"].label = 'vc - RL Element'
ts_dpsim_dp_emt2["l1.i_intf_shift"].label = 'i - RL Element'

pt.plot_timeseries(2, ts_dpsim_dp_emt["n3.v_shift"])
pt.plot_timeseries(2, ts_dpsim_dp_emt2["n2.v_shift"])

pt.plot_timeseries(3, ts_dpsim_dp_emt["l1.i_intf_shift"])
pt.plot_timeseries(3, ts_dpsim_dp_emt2["l1.i_intf_shift"])

### Assert

In [ ]:
# calculate the RMSE between both models
err_current = ts.rmse(ts_dpsim_dp_emt["l1.i_intf_shift"], ts_dpsim_dp_emt2["l1.i_intf_shift"])
err_voltage = ts.rmse(ts_dpsim_dp_emt["n3.v_shift"], ts_dpsim_dp_emt2["n2.v_shift"])

assert err_current < 0.00001
assert err_voltage < 0.00001